In [1]:
import numpy as  np

In [2]:
entropies = [0.0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

data_pl = {} # e -> [exit_layer_count, eval_time, actual_cost/full_cost, accuracy]
for e in entropies:
    data_pl[e] = np.load('./../plotting2/saved_models/bert_base-SST-2-two_stage/entropy_{}.npy'.format(e), allow_pickle=True)

In [3]:
acc_buckets = {}
# for acc in range(60, 100, 5):
#     acc_buckets[acc] = []

for e in entropies:
    for exit_layer in data_pl[e][3]:
        acc = int(data_pl[e][3][exit_layer] * 20) * 5
        if acc >= 60:
            if acc not in acc_buckets:
                acc_buckets[acc] = []
            latency = int(data_pl[e][1][exit_layer] * 1000)
            acc_buckets[int(acc/5)*5].append((latency, exit_layer, e))

for acc in acc_buckets:
    print(acc, sorted(acc_buckets[acc]))


90 [(21, 1, 0.5), (32, 2, 0.3), (35, 2, 0.2), (48, 3, 0.05), (92, 6, 0.15), (96, 6, 0.2), (99, 6, 0.05), (102, 7, 0.001), (111, 7, 0.15), (111, 7, 0.3), (120, 8, 0.15), (123, 8, 0.1), (128, 8, 0.2), (177, 12, 0.0)]
100 [(30, 2, 0.01), (32, 2, 0.001), (35, 2, 0.005), (37, 2, 0.05), (47, 3, 0.005), (47, 3, 0.01), (62, 4, 0.005), (65, 4, 0.05), (72, 4, 0.001), (74, 5, 0.005), (78, 5, 0.1), (79, 5, 0.01), (80, 5, 0.001), (80, 5, 0.05), (99, 6, 0.001), (105, 7, 0.005), (119, 8, 0.6), (128, 9, 0.6), (154, 11, 0.15), (156, 11, 0.3), (158, 11, 0.4), (159, 11, 0.001), (175, 11, 0.1), (219, 12, 0.5)]
85 [(16, 1, 0.6), (32, 2, 0.4), (34, 2, 0.5), (47, 3, 0.3), (50, 3, 0.4), (61, 4, 0.15), (64, 4, 0.1), (67, 4, 0.2), (79, 5, 0.3), (87, 5, 0.2), (91, 6, 0.5), (95, 6, 0.4), (95, 6, 0.6), (102, 6, 0.3), (102, 7, 0.4), (113, 7, 0.2), (113, 7, 0.6), (120, 8, 0.4), (122, 8, 0.3), (144, 10, 0.01), (147, 9, 0.05), (157, 11, 0.01), (175, 12, 0.001), (179, 11, 0.05)]
95 [(16, 1, 0.4), (31, 2, 0.1), (31, 2, 

In [6]:
# Accuracy is prioritized. We acheive the highest accuracy possible within the latency budget.
def get_entropies(targt_accuracy, target_latency):
    entropies = [(0, 0, 10)] * 12 # e, acc, latency
    for acc in acc_buckets:
        if acc < targt_accuracy:
            continue
        for latency, exit_layer, e in acc_buckets[acc]:
            if latency > target_latency:
                continue
            if entropies[exit_layer][1] < acc or (entropies[exit_layer][1] == acc and entropies[exit_layer][2] > latency):
                entropies[exit_layer] = (e, acc, latency)
   
    res = []
    for e, _, _ in entropies:
        res.append(e)
    
    for i in range(len(res) - 1, 0, -1):
        if res[i] != 0:
            res[i] = 1
            break
    return res

print("for 90% acc, 150ms latency: ", get_entropies(90, 150))
print("for 80% acc, 150ms latency: ", get_entropies(80, 150))
print("for 80% acc, 120ms latency: ", get_entropies(80, 120))
print("for 70% acc, 120ms latency: ", get_entropies(70, 120))
print("for 70% acc, 100ms latency: ", get_entropies(70, 100))
            
            

for 90% acc, 150ms latency:  [0, 0.4, 0.01, 0.005, 0.005, 0.005, 0.001, 0.005, 0.6, 0.6, 1, 0]
for 80% acc, 150ms latency:  [0, 0.4, 0.01, 0.005, 0.005, 0.005, 0.001, 0.005, 0.6, 0.6, 1, 0]
for 80% acc, 120ms latency:  [0, 0.4, 0.01, 0.005, 0.005, 0.005, 0.001, 0.005, 1, 0, 0, 0]
for 70% acc, 120ms latency:  [0, 0.4, 0.01, 0.005, 0.005, 0.005, 0.001, 0.005, 1, 0, 0, 0]
for 70% acc, 100ms latency:  [0, 0.4, 0.01, 0.005, 0.005, 0.005, 1, 0, 0, 0, 0, 0]
